In [3]:
from typing import Optional, List
from pydantic import BaseModel, Field
from enum import Enum

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

In [2]:
class ScenarioType(str, Enum):
    """Type of collection call scenario."""
    PTP = "PTP"
    REFUSE_TO_PAY = "REFUSE_TO_PAY"
    TPC = "TPC"
    UNKNOWN = "UNKNOWN"

class Amount(BaseModel):
    """Amount mentioned in the conversation."""
    value: float = Field(..., description="The monetary value")
    currency: str = Field(default="IDR", description="Currency of the amount")
    type: str = Field(..., description="Type of amount (e.g., installment, penalty, total)")

class BasicCallInfo(BaseModel):
    """Basic information extracted from a collection call."""
    agent_name: Optional[str] = Field(
        default=None, 
        description="Name of the collection agent"
    )
    customer_name: Optional[str] = Field(
        default=None, 
        description="Name of the customer being contacted"
    )
    scenario_type: ScenarioType = Field(
        default=ScenarioType.UNKNOWN,
        description="Type of collection scenario identified"
    )
    call_duration: Optional[str] = Field(
        default=None,
        description="Duration of the call in timestamp format"
    )
    amounts_mentioned: List[Amount] = Field(
        default_factory=list,
        description="List of monetary amounts mentioned in the call"
    )
    payment_date_mentioned: Optional[str] = Field(
        default=None,
        description="Any payment date mentioned in the conversation"
    )

class PTPDetails(BaseModel):
    """Additional details specific to PTP (Promise to Pay) scenario."""
    promised_date: Optional[str] = Field(
        default=None,
        description="Date customer promised to pay"
    )
    promised_amount: Optional[Amount] = Field(
        default=None,
        description="Amount customer promised to pay"
    )
    negotiation_attempts: Optional[int] = Field(
        default=None,
        description="Number of times agent attempted to negotiate payment"
    )

class RefuseDetails(BaseModel):
    """Additional details specific to Refuse to Pay scenario."""
    reason: Optional[str] = Field(
        default=None,
        description="Main reason given for refusing payment"
    )
    customer_situation: Optional[str] = Field(
        default=None,
        description="Description of customer's stated situation or obstacles"
    )

class TPCDetails(BaseModel):
    """Additional details specific to Third Party Contact scenario."""
    relationship_to_customer: Optional[str] = Field(
        default=None,
        description="Relationship of the contacted person to the customer"
    )
    message_delivered: Optional[bool] = Field(
        default=None,
        description="Whether a message was successfully delivered"
    )

class CallData(BaseModel):
    """Complete extraction of collection call data."""
    basic_info: BasicCallInfo
    ptp_details: Optional[PTPDetails] = Field(
        default=None,
        description="Details specific to PTP scenario, if applicable"
    )
    refuse_details: Optional[RefuseDetails] = Field(
        default=None,
        description="Details specific to Refuse to Pay scenario, if applicable"
    )
    tpc_details: Optional[TPCDetails] = Field(
        default=None,
        description="Details specific to Third Party Contact scenario, if applicable"
    )
    call_summary: Optional[str] = Field(
        default=None,
        description="Brief summary of the key points from the call"
    )

# Prompt template for the extraction
prompt_template = """
You are an expert collection call analyzer. Your task is to extract relevant information from collection call transcripts in Indonesian language.

Key things to look for:
1. Identify the type of call scenario (PTP, Refuse to Pay, or TPC)
2. Extract basic call information
3. Extract scenario-specific details
4. Create a brief summary

Remember:
- Only extract information that is explicitly present in the transcript
- Return null for any information that cannot be confidently extracted
- Pay attention to Indonesian language patterns and context
- Look for monetary amounts, dates, and key phrases that indicate the scenario type

Transcript to analyze:
{text}
"""

In [4]:
from dotenv import load_dotenv

load_dotenv("../.env")

True

In [10]:
llm = ChatOpenAI(model="gpt-4o")  # or your preferred model
structured_llm = llm.with_structured_output(schema=CallData)
prompt = ChatPromptTemplate.from_messages([
    ("system", prompt_template),
    ("human", "{text}")
])

In [11]:
def analyze_transcript(transcript_text):
    prompt_value = prompt.invoke({"text": transcript_text})
    return structured_llm.invoke(prompt_value)

In [12]:
text = """
[00:00.000 --> 00:25.400]  Halo selamat pagi Rp. 4.000, Bapak. Untuk angkusan BV-nya, apakah sudah bisa dibayarkannya hari ini, Bapak Jalaluddin?
[00:29.440 --> 00:30.100]  Halo, Pak?
[00:30.120 --> 00:31.140]  Ini dari mana ini?
[00:31.560 --> 00:36.340]  Dari mana? Dari BV Finance, Pak. Ini untuk angkusan Mitsubishi Cloud-nya, Pak, yang warna kuning.
[00:36.900 --> 00:40.080]  Untuk angkusan BV-nya, apakah sudah bisa dibayarkannya hari ini, Bapak Jalaluddin?
[00:48.160 --> 00:53.800]  Tapi kalau nggak ada kopnya B mana-mana namanya namanya gimana maksudnya ini dengan Bapak jalurin kan Pak
[00:53.800 --> 01:07.380]  saya kena bilang saya novel dari BV Finance saya novel dari BV Finance Pak Angson BFI nya Bapak sudah Oh setiap yang nelfon ganti-ganti kenapa?
[01:07.640 --> 01:09.420]  Iya ini tau-tau telfonnya by system Pak
[01:09.420 --> 01:12.440]  Ini Angson BFI nya sudah ketambahannya 1 hari
[01:12.440 --> 01:14.220]  Dengan nominal yang harus dibayarkan disini
[01:14.220 --> 01:17.060]  Rp. 6.364.000
[01:17.060 --> 01:20.540]  Angson BFI nya Pak apakah sudah bisa dibayar hari ini?
[01:21.000 --> 01:21.920]  Bapak Jalaludin
[01:21.920 --> 01:25.880]  Saya belum bisa masjidkan
[01:25.880 --> 01:27.380]  kenapa? kendalanya apa?
[01:28.280 --> 01:30.220]  hujan, gak bisa panen
[01:30.220 --> 01:32.060]  untuk kejadian ini kan juga bukan
[01:32.060 --> 01:33.700]  kejadian yang pertama Pak, dan Bapak juga bisa
[01:33.700 --> 01:35.620]  antisipasi juga sebelumnya, karena disini
[01:35.620 --> 01:37.840]  untuk tanggal jatuh tembani Bapak sendiri juga disini
[01:37.840 --> 01:46.780]  tidak akan menabur Bapak setiap bulannya karena jika pembayarannya Bapak di sini bagus, saat pengajuannya kembali juga nantinya akan dipermudah juga. Jadi sekarang bisa dibayarkan dua kali kan di tanggal berapa?
[01:46.820 --> 01:47.420]  Iya, kamu bicara banyak.
[01:47.900 --> 01:48.680]  Di tanggal berapa?
[01:48.780 --> 01:49.340]  Saya tidak tahu.
[01:49.660 --> 01:51.000]  Di tanggal berapa bisa dibayar?
[01:51.020 --> 01:52.800]  Kalau menunggu hujan, travel alam.
[01:54.400 --> End Recording]  Ini sudah ketempatan satu hari Pak, jika pembayarannya selalu tidak tepat waktu, maka
"""

In [14]:
response = analyze_transcript(text)

In [17]:
response.model_dump_json()

'{"basic_info":{"agent_name":"Novel","customer_name":"Bapak Jalaluddin","scenario_type":"REFUSE_TO_PAY","call_duration":"[00:00.000 --> 01:54.400]","amounts_mentioned":[{"value":6364000.0,"currency":"IDR","type":"installment"}],"payment_date_mentioned":null},"ptp_details":null,"refuse_details":{"reason":"hujan, gak bisa panen","customer_situation":"Bapak Jalaluddin tidak bisa membayar karena cuaca hujan yang menghambat panen."},"tpc_details":null,"call_summary":"Agent Novel from BV Finance contacted Bapak Jalaluddin regarding an overdue installment payment of IDR 6,364,000. Bapak Jalaluddin refused to pay, citing rain that prevented harvesting as the reason for his inability to pay. The agent attempted to negotiate a payment date but was unsuccessful."}'

---

In [26]:
from typing import Optional, List, Dict
from pydantic import BaseModel, Field
from enum import Enum

# [Previous Enum and Model definitions remain the same...]
# Basic Enums and Types through Complete QA Score Model stay unchanged

# Updated Scoring Prompt Template with proper variable handling
SCORING_ANALYSIS_PROMPT = """You are an expert QA analyst for collection call centers, specializing in evaluating agent performance based on strict QA criteria. Your task is to analyze a call transcript and extract detailed scoring information.

Context: The call has already been classified as {scenario_type}. You will evaluate the call based on the appropriate QA form criteria for this scenario type.

Required Output Format:
The output should be a JSON object matching the following structure example:
{{
    "scenario_type": "{scenario_type}",
    "opening_score": {{
        "greeting_score": "0|0.5|1",
        "greeting_evidence": "exact quote from transcript",
        "customer_name_verification": "COMPLIANT|NON_COMPLIANT|NOT_APPLICABLE",
        "customer_verification_evidence": "exact quote or null",
        "mandatory_info_disclosed": ["list of disclosed items"]
    }},
    "communication_score": {{
        "voice_tone_score": "0|0.5|1",
        "voice_tone_evidence": "evidence or null",
        "speaking_pace_score": "0|0.5|1",
        "speaking_pace_evidence": "evidence or null",
        "language_etiquette_score": "0|0.5|1",
        "language_evidence": ["examples"]
    }},
    "negotiation_score": {{
        "negotiation_attempts": number,
        "solutions_offered": ["list of solutions"],
        "payment_commitment_obtained": boolean,
        "negotiation_evidence": ["key phrases"]
    }},
    "knockout_violations": {{
        "unauthorized_disclosure": boolean,
        "disclosure_evidence": "evidence or null",
        "ptp_cheating": boolean,
        "ptp_cheating_evidence": "evidence or null",
        "other_violations": ["list of violations"]
    }},
    "total_score": number between 0-1,
    "score_breakdown": {{
        "opening": number,
        "communication": number,
        "negotiation": number
    }},
    "improvement_areas": ["list of areas"],
    "evidence_highlights": ["list of key evidence"]
}}

Evaluation Guidelines:

1. Opening Section (6% weight):
- Listen for proper greeting, agent name, and company name
- Verify correct customer name usage
- Check for mandatory information disclosure
- Score: 0 (non-compliant), 0.5 (standard), 1 (strong)

2. Communication Skills (25% total weight):
- Voice tone (6%): Energy, professionalism, confidence
- Speaking pace (6%): Clear articulation, appropriate speed
- Language etiquette (13%): Politeness, appropriate phrases
- Score each component: 0 (poor), 0.5 (acceptable), 1 (excellent)

3. Negotiation Skills (40% total weight, for PTP/RTP):
- Track negotiation attempts
- Document solutions offered
- Verify payment commitments
- Evaluate effectiveness: 0 (ineffective), 0.5 (moderate), 1 (highly effective)

4. Knockout Criteria (Immediate Fail):
- Information disclosure to unauthorized parties
- PTP cheating
- Policy violations
- Document any violations with specific evidence

Important:
- Provide specific evidence from the transcript for each score
- Score breakdown must add up to total score
- All mandatory fields must be included in the output
- Use exact quotes when providing evidence

Call Transcript to analyze:
{text}

Analyze the transcript and provide a complete evaluation following the exact structure shown above."""

def create_scenario_specific_prompt(scenario_type: str, base_prompt: str) -> str:
    """Customize scoring prompt based on scenario type"""
    
    scenario_additions = {
        "PTP": """
Additional PTP Criteria:
- Focus on commitment clarity
- Verify payment amount and date
- Check for proper confirmation
- Evaluate follow-up scheduling""",
        
        "REFUSE_TO_PAY": """
Additional RTP Criteria:
- Evaluate reason documentation
- Check solution exploration
- Assess escalation handling
- Monitor professional persistence""",
        
        "TPC": """
Additional TPC Criteria:
- Verify relationship confirmation
- Check information protection
- Evaluate message clarity
- Assess contact information gathering"""
    }
    
    return base_prompt + scenario_additions.get(scenario_type, "")

In [24]:
class QAScoringAnalyzer:
    """Handles QA scoring analysis using LangChain"""
    
    def __init__(self, model_name: str = "gpt-3.5-turbo", temperature: float = 0):
        self.llm = ChatOpenAI(
            model=model_name,
            temperature=temperature
        )
        self.structured_llm = self.llm.with_structured_output(QAScore)
        
    def analyze_call(self, 
                    transcript: str, 
                    scenario_type: ScenarioType,
                    custom_prompt: Optional[str] = None) -> QAScore:
        """Analyze a call transcript and return structured QA scoring"""
        
        # Create scenario-specific prompt
        base_prompt = custom_prompt or SCORING_ANALYSIS_PROMPT
        final_prompt = create_scenario_specific_prompt(scenario_type, base_prompt)
        
        # Create prompt template
        prompt = ChatPromptTemplate.from_messages([
            ("system", final_prompt),
            ("human", "{text}")
        ])
        
        # Generate prompt values
        prompt_value = prompt.invoke({
            "text": transcript,
            "scenario_type": scenario_type
        })
        
        # Get structured output
        try:
            result = self.structured_llm.invoke(prompt_value)
            return result
        except Exception as e:
            raise Exception(f"Error during QA scoring analysis: {str(e)}")

In [27]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from typing import Optional

class QAScoringAnalyzer:
    """Handles QA scoring analysis using LangChain"""
    
    def __init__(self, model_name: str = "gpt-3.5-turbo", temperature: float = 0):
        self.llm = ChatOpenAI(
            model=model_name,
            temperature=temperature
        )
        self.structured_llm = self.llm.with_structured_output(QAScore)
        
    def analyze_call(self, 
                    transcript: str, 
                    scenario_type: ScenarioType) -> QAScore:
        """Analyze a call transcript and return structured QA scoring"""
        
        # Create prompt template with proper escaping
        prompt = ChatPromptTemplate.from_template(SCORING_ANALYSIS_PROMPT)
        
        try:
            # Generate prompt values
            prompt_value = prompt.invoke({
                "text": transcript,
                "scenario_type": scenario_type.value
            })
            
            # Get structured output
            result = self.structured_llm.invoke(prompt_value)
            
            # Validate score_breakdown
            if not result.score_breakdown:
                result.score_breakdown = {
                    "opening": 0.0,
                    "communication": 0.0,
                    "negotiation": 0.0
                }
                
            return result
            
        except Exception as e:
            print(f"Raw error: {str(e)}")  # For debugging
            raise Exception(f"Error during QA scoring analysis: {str(e)}")

# Example usage
def test_qa_scoring():
    # Test transcript
    test_transcript = """
    Agent: Selamat pagi, saya John dari BFI Finance. Bisa bicara dengan Bapak Ahmad?
    Customer: Ya, saya sendiri.
    Agent: Pak Ahmad, saya menghubungi terkait pembayaran angsuran yang sudah jatuh tempo...
    """
    
    # Initialize analyzer
    analyzer = QAScoringAnalyzer()
    
    # Analyze call
    try:
        qa_score = analyzer.analyze_call(
            transcript=text,
            scenario_type=ScenarioType.PTP
        )
        
        # Access results
        print(f"Total Score: {qa_score.total_score}")
        print(f"Score Breakdown: {qa_score.score_breakdown}")
        print(f"Improvement Areas: {qa_score.improvement_areas}")
        
        # Check for knockout violations
        if qa_score.knockout_violations.unauthorized_disclosure:
            print("WARNING: Unauthorized information disclosure detected")
            print(f"Evidence: {qa_score.knockout_violations.disclosure_evidence}")
            
    except Exception as e:
        print(f"Analysis failed: {str(e)}")

if __name__ == "__main__":
    test_qa_scoring()

Total Score: 0.625
Score Breakdown: {'opening': 0.0, 'communication': 0.0, 'negotiation': 0.0}
Improvement Areas: ['Improve speaking pace for better clarity.', 'Enhance negotiation skills to obtain payment commitment.']
